# Clasificación de textos

En esta clase, exploraremos datos de mensajes de texto y crearemos modelos para predecir si un mensaje es spam o no.

In [2]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [4]:
spam_data['target_text']= spam_data['target'].apply(lambda x: 'spam' if x else 'no_spam')
spam_data.head(10)

,text,target,target_text
0,"Go until jurong point, crazy.. Available only ...",0,no_spam
1,Ok lar... Joking wif u oni...,0,no_spam
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,spam
3,U dun say so early hor... U c already then say...,0,no_spam
4,"Nah I don't think he goes to usf, he lives aro...",0,no_spam
5,FreeMsg Hey there darling it's been 3 week's n...,1,spam
6,Even my brother is not like to speak with me. ...,0,no_spam
7,As per your request 'Melle Melle (Oru Minnamin...,0,no_spam
8,WINNER!! As a valued network customer you have...,1,spam
9,Had your mobile 11 months or more? U R entitle...,1,spam


In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [99]:
print(X_train.shape, y_train.shape)

(4179,) (4179,)


### Pregunta 1
¿Qué porcentaje de los documentos en `spam_data` son spam?

*Esta función debe devolver un valor flotante, el valor porcentual (es decir, $ ratio * 100 $).*

In [3]:
def respuesta_uno():
    spam_data['target'].mean()    #Calcula el promedio de mesajes spam, pero no lo usa(?)
    spam=spam_data[spam_data['target'] != 0]    #Obtiene los elementos que son spam
    print(len(spam), len(spam_data))    #Imprime la relación entre mensaje totales y spam
    return spam_data['target'].mean()*100   #Regresa el promedio de datos (porcentaje de documentos)

In [4]:
respuesta_uno()

(747, 5572)


13.406317300789663

### Pregunta 2

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

Luego, ajuste un modelo de clasificación Naive Bayes multinomial. Calcule medidas de exactitud, presición, recall y f1-score usando los datos de prueba transformados.

*Esta función debe devolver las cuatro medidas de evaluación como una lista con los 4 valores en el orden solicitado cada valor como flotante.*

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def respuesta_dos():
    scores=[]
    
    vect = CountVectorizer().fit(X_train) #Learn a vocabulary dictionary of all tokens in the raw documents.
    X_train_vectorized=vect.transform(X_train)  #Transform documents to document-term matrix.
    
    clf= MultinomialNB()  #Naive Bayes classifier for multinomial models.
    clf.fit( X_train_vectorized, y_train ) #Fit Naive Bayes classifier according to X, y.
    print("matriz Frecuencias shape:",X_train_vectorized.shape, "num clases",len(y_train))
    
    X_test_vectorized= vect.transform( X_test )
    predictions = clf.predict(X_test_vectorized)
    print("matriz Frecuencias Test shape:",X_test_vectorized.shape, "num prediction",len(predictions))
    
    #Medidas de evaluación
    scores.append(accuracy_score(y_test,predictions))
    scores.append(precision_score(y_test,predictions))
    scores.append(recall_score(y_test,predictions))
    scores.append(f1_score(y_test,predictions))
    
    #Imprime los valores de la predicción de los textos de spam
    spam=predictions[predictions!= 0]
    print('num Predicciones Spam:',len(spam))
    print('Porcentaje Spam:',(len(spam)/len(predictions))*100)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(confusion_matrix(y_test, predictions))
    print(tn, fp, fn, tp)
    return scores

In [10]:
respuesta_dos()

('matriz Frecuencias:', (4179, 7354), 'clases', 4179)
('matriz Frecuencias Test:', (1393, 7354), 'prediction', 1393)
('Predicciones Spam:', 184)
('Porcentaje Spam:', 0)
[[1193    3]
 [  16  181]]
(1193, 3, 16, 181)


[0.9863603732950467,
 0.9836956521739131,
 0.9187817258883249,
 0.9501312335958005]

### Pregunta 3

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

Luego, ajuste un modelo de clasificación Naive Bayes multinomial con suavizado (smoothing) `alpha = 0.1`. Encuentre el área bajo la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def respuesta_tres():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    clf=MultinomialNB(alpha=0.1)
    clf.fit(X_train_vectorized, y_train)
    X_test_vectorized=vect.transform(X_test)
    predictions = clf.predict(X_test_vectorized)
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [12]:
respuesta_tres()

[0 0 0 ... 0 0 1]


0.9720812182741116

### Pregunta 4

Ajustar y transformar los datos de entrenamiento `X_train` utilizando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **3**.

Luego, ajuste un modelo de clasificador Naive Bayes multinomial con suavizado (smoothing) `alfa = 0.1` y calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def respuesta_cuatro():
    #Genero un objeto vectorizer y Learn vocabulary and idf from training set.
    vectorizer= TfidfVectorizer( min_df=3 ).fit( X_train )
    #Transform documents to document-term matrix.
    X_train_vectorizer= vectorizer.transform( X_train )
    
    #Generación de modelo Naive-Bayes con suavizado y ajuste con los datos generados arriba
    clf= MultinomialNB(alpha=0.1)
    clf.fit( X_train_vectorizer, y_train )
    
    #Predicciones, obtenemos la matriz de terminos y se lo pasamos al modelo
    X_test_vectorizerd= vectorizer.transform( X_test )
    predictions= clf.predict( X_test_vectorizerd )
    
    
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [91]:
respuesta_cuatro()

[0 0 0 ... 0 0 1]


0.9416243654822335

### Pregunta 5

¿Cuál es la longitud promedio de los documentos (número de caracteres) para los documentos spam y no spam?

*Esta función debe devolver una tupla (longitud promedio no spam, longitud promedio de spam).*

In [17]:
def respuesta_cinco():
    return spam_data[ spam_data['target'] == 1 ]['text'].str.count('\w').mean(), spam_data[ spam_data['target'] == 0 ]['text'].str.count('\w').mean()
    

In [18]:
respuesta_cinco()

(109.82463186077644, 53.73181347150259)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [19]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Pregunta 6

Ajustar y transformar los datos de entrenamiento `X_train` usando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5**.

Usando esta matriz de término de documento y una característica adicional, **la longitud del documento (número de caracteres)**, ajustar a un modelo de Clasificación de Vector de Soporte con regularización `C = 10000`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [104]:
from sklearn.svm import SVC

def respuesta_seis():
    # to count the number of words
    #cv= CountVectorizer() 
    #word_count_vector= cv.fit_transform( X_train )
    #count_vector= cv.transform( X_train )
    #vectorizer= TfidfVectorizer( min_df=5 ).fit( word_count_vector )
    #X_train_vectorizer= vectorizer.transform( count_vector )
    
    #Genero un objeto vectorizer y Learn vocabulary and idf from training set.
    vectorizer= TfidfVectorizer( min_df=3 ).fit( X_train )
    #Transform documents to document-term matrix.
    X_train_vectorizer= vectorizer.transform( X_train )
    #Añadimos una nueva caracteristica
    X_train_vectorizer= add_feature( X_train_vectorizer, X_train.str.count('\w') )
    
    #Creamos el modelo y entrenamos
    clf= SVC( C= 10000 )
    #clf = make_pipeline(StandardScaler(), SVC(C=10000))
    clf.fit( X_train_vectorizer, y_train )

    #Predicciones, obtenemos la matriz de terminos y se lo pasamos al modelo
    X_test_vectorizerd= vectorizer.transform( X_test )
    X_test_vectorizerd= add_feature( X_test_vectorizerd, X_test.str.count('\w') )
    
    
    predictions= clf.predict( X_test_vectorizerd )
    
    
    print(predictions)
    return roc_auc_score(y_test,predictions)



In [105]:
respuesta_seis()

[0 0 0 ... 0 0 1]


0.9657508955401253

### Pregunta 7

¿Cuál es el número promedio de dígitos por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # dígitos no es spam, promedio # dígitos spam).*

In [106]:
def respuesta_siete():
    return spam_data[ spam_data['target'] == 1 ]['text'].str.count('\d').mean(), spam_data[ spam_data['target'] == 0 ]['text'].str.count('\d').mean()

In [107]:
respuesta_siete()

(15.759036144578314, 0.2992746113989637)

### Pregunta 8

Ajustar y transformar los datos de entrenamiento `X_train` usando un `TfidfVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5** y usando **n-gramas de palabras n = 1 a n = 3** (unigramas, bigramas y trigramas).

Usando esta matriz de término-documento y las siguientes características adicionales:
* la longitud del documento (número de caracteres)
* **cantidad de dígitos por documento**

Ajustar un modelo de Regresión logística con regularización `C = 100`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

*Esta función debe devolver el AUC como un flotante.*

In [108]:
from sklearn.linear_model import LogisticRegression

def respuesta_ocho():
    #Genero un objeto vectorizer y Learn vocabulary and idf from training set.
    vectorizer= TfidfVectorizer( ngram_range=(1,3), min_df=5 ).fit( X_train )
    #Transform documents to document-term matrix.
    X_train_vectorizer= vectorizer.transform( X_train )
    #Añadimos nuevas caracteristica
    X_train_vectorizer= add_feature( X_train_vectorizer, X_train.str.count('\w') )
    X_train_vectorizer= add_feature( X_train_vectorizer, X_train.str.count('\d') )
    
    #Creamos el modelo y entrenamos
    clf= LogisticRegression( C= 100 )
    clf.fit( X_train_vectorizer, y_train )

    #Predicciones, obtenemos la matriz de terminos y se lo pasamos al modelo
    X_test_vectorizerd= vectorizer.transform( X_test )
    X_test_vectorizerd= add_feature( X_test_vectorizerd, X_test.str.count('\w') )
    X_test_vectorizerd= add_feature( X_test_vectorizerd, X_test.str.count('\d') )
    predictions= clf.predict( X_test_vectorizerd )
    
    
    print(predictions)
    return roc_auc_score(y_test,predictions)

In [109]:
respuesta_ocho()

[0 0 0 ... 0 0 1]


/Users/aceron/opt/miniconda3/envs/maestria/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9708270376721049

### Pregunta 9

¿Cuál es el número promedio de caracteres que no son palabras (cualquier cosa que no sea una letra, un dígito o un guión bajo) por documento para los documentos que no son spam y spam?

*Sugerencia: utilice las clases de caracteres `\ w` y` \ W`*

*Esta función debe devolver una tupla (promedio de # caracteres que no son palabras, no spam, promedio de # caracteres que no son palabras, spam).*

In [110]:
def pregunta_nueve():
        return spam_data[ spam_data['target'] == 1 ]['text'].str.count('\W').mean(), spam_data[ spam_data['target'] == 0 ]['text'].str.count('\W').mean()

In [111]:
pregunta_nueve()

(29.041499330655956, 17.29181347150259)

### Pregunta 10

Ajustar y transformar los datos de entrenamiento `X_train` usando un `CountVectorizer` ignorando los términos que tienen una frecuencia de documento estrictamente inferior a **5** y utilizando **caracteres n-grams desde n = 2 a n = 5.**

Para decirle a `CountVectorizer` que use caracteres n-grams, pase en `analyzer = 'char_wb'` que crea caracteres n-gramas solo del texto dentro de los límites de las palabras. Esto debería hacer que el modelo sea más robusto a los errores ortográficos.

Usando esta matriz término documento y las siguientes características adicionales:
* la longitud del documento (número de caracteres)
* cantidad de dígitos por documento
* **cantidad de caracteres que no son palabras (cualquier cosa que no sea una letra, dígito o guión bajo).**

Ajustar un modelo de Regresión logística con regularización `C = 100`. Luego calcule el área bajo de la curva (AUC) usando los datos de prueba transformados.

También **encuentre los 10 coeficientes más pequeños y los 10 más grandes del modelo** y devuélvalos junto con el AUC en una tupla.

La lista de los 10 coeficientes más pequeños debe ordenarse primero, la lista de los 10 coeficientes más grandes debe ordenarse primero.

Las tres características que se agregaron a la matriz de términos del documento deben tener los siguientes nombres si aparecen en la lista de coeficientes:
['longitud_doc', 'conteo_digito', 'caracteres_no_palabra']

*Esta función debe devolver una tupla `(AUC como flotante, lista de coeficientes más pequeños, lista de coeficientes más grande)`.*

In [133]:
from sklearn.linear_model import LogisticRegression

def respuesta_10():
    vect = CountVectorizer( analyzer = 'char_wb', ngram_range=(2,5), max_df=5 ).fit(X_train) #Learn a vocabulary dictionary of all tokens in the raw documents.
    X_train_vectorizer=vect.transform(X_train)  #Transform documents to document-term matrix.
    #Añadimos nuevas caracteristica
    X_train_vectorizer= add_feature( X_train_vectorizer, X_train.str.count('\W') )
    X_train_vectorizer= add_feature( X_train_vectorizer, X_train.str.count('\w') )
    X_train_vectorizer= add_feature( X_train_vectorizer, X_train.str.count('\d') )
    
    clf= LogisticRegression( C=100 )  #Naive Bayes classifier for multinomial models.
    clf.fit( X_train_vectorizer, y_train ) #Fit Naive Bayes classifier according to X, y.
    
    X_test_vectorizerd= vect.transform( X_test )
    X_test_vectorizerd= add_feature( X_test_vectorizerd, X_test.str.count('\W') )
    X_test_vectorizerd= add_feature( X_test_vectorizerd, X_test.str.count('\w') )
    X_test_vectorizerd= add_feature( X_test_vectorizerd, X_test.str.count('\d') )
    predictions = clf.predict(X_test_vectorizerd)
    
    coef= clf.coef_
    lowr= sorted(coef)[:10]
    uppr= sorted(coef,reverse=True)
    
    scores=[]
    #Medidas de evaluación
    scores.append(accuracy_score(y_test,predictions))
    scores.append(precision_score(y_test,predictions))
    scores.append(recall_score(y_test,predictions))
    scores.append(f1_score(y_test,predictions))
    
    #Imprime los valores de la predicción de los textos de spam
    spam=predictions[predictions!= 0]
    print('num Predicciones Spam:',len(spam))
    print('Porcentaje Spam:',(len(spam)/len(predictions))*100)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(confusion_matrix(y_test, predictions))
    print(tn, fp, fn, tp)
    
    
    return scores, (lowr, uppr)

In [134]:
respuesta_10()

num Predicciones Spam: 186
Porcentaje Spam: 13.352476669059584
[[1191    5]
 [  16  181]]
1191 5 16 181


/Users/aceron/opt/miniconda3/envs/maestria/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


([0.9849246231155779,
  0.9731182795698925,
  0.9187817258883249,
  0.9451697127937337],
 ([array([-0.0177898 , -0.01550143, -0.00228836, ..., -0.42850894,
           0.14614202,  1.64518111])],
  [array([-0.0177898 , -0.01550143, -0.00228836, ..., -0.42850894,
           0.14614202,  1.64518111])]))